In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt
import torch 
import torch.nn as nn
import yfinance as yf
import keras 
import os
import tensorflow
import datetime 
import requests
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras import optimizers
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [3]:
#takes any ticker and returns X as past # days and Y as tmr's price, split into train and test
def DataPrep(ticker='aapl',daysago=100,split=0.9):
    df = yf.Ticker(ticker.upper()).history(period='max')
    if df.shape[1] == 7:
        df = df.drop(columns=['Dividends','Stock Splits'])
    df = df.drop(columns=['Open','High','Low','Volume'])
    for i in range(daysago,-2,-1):
        df['{}daysago'.format(i)] = df['Close'].shift(i)
        df = df.copy()
    df = df.drop(columns=['Close'])
    df = df.dropna()
    sc = MinMaxScaler(feature_range=(0,1))
    df_scaled = sc.fit_transform(df)
    X,Y = df_scaled[:,:daysago+1],df_scaled[:,-1]
    split = int(len(X)*split)
    X_train, X_test, Y_train, Y_test = X[:split], X[split:], Y[:split], Y[split:]
    X_train, X_test = X_train.reshape((-1,daysago+1,1)), X_test.reshape((-1,daysago+1,1))
    Y_train, Y_test = Y_train.reshape((-1,1)), Y_test.reshape((-1,1))
    return X_train, X_test, Y_train, Y_test

#takes any list of tickers, compiles x_train, x_test, y_train, y_test, @ a set daysago and split
def CompileData(tickers=['aapl','nvda'],daysago=100,split=0.9):
    X_train,X_test,Y_train,Y_test = [],[],[],[]
    X_train,X_test,Y_train,Y_test = np.array(X_train).reshape(-1,daysago+1,1),np.array(X_test).reshape(-1,daysago+1,1), np.array(Y_train).reshape(-1,1), np.array(Y_test).reshape(-1,1)
    for ticker in tickers:
        dataprep = DataPrep(ticker,daysago,split)
        X_train = np.append(X_train,dataprep[0],axis=0)
        X_test = np.append(X_test,dataprep[1],axis=0)
        Y_train = np.append(Y_train,dataprep[2],axis=0)
        Y_test = np.append(Y_test,dataprep[3],axis=0)
    return X_train, X_test, Y_train, Y_test
CompileData()

(array([[[3.09577038e-04],
         [2.83417097e-04],
         [2.48533581e-04],
         ...,
         [2.51314495e-04],
         [2.40308723e-04],
         [2.45811625e-04]],
 
        [[2.83417097e-04],
         [2.48533581e-04],
         [2.59433515e-04],
         ...,
         [2.40308723e-04],
         [2.45811625e-04],
         [2.49480110e-04]],
 
        [[2.48533581e-04],
         [2.59433515e-04],
         [2.72517314e-04],
         ...,
         [2.45811625e-04],
         [2.49480110e-04],
         [2.40308723e-04]],
 
        ...,
 
        [[2.41532410e-01],
         [2.41047347e-01],
         [2.45865133e-01],
         ...,
         [1.71949296e-01],
         [1.71934550e-01],
         [1.64214660e-01]],
 
        [[2.41047347e-01],
         [2.45865133e-01],
         [2.43914318e-01],
         ...,
         [1.71934550e-01],
         [1.64214660e-01],
         [1.67514764e-01]],
 
        [[2.45865133e-01],
         [2.43914318e-01],
         [2.40432973e-01],
         

In [23]:
def ReadIndex(filename='SP500',blacklist=['BRK.B','BF.B','WRK']):
    df = pd.read_csv('{}.csv'.format(filename))
    ListOfTickers = df['Symbol'].tolist()
    for ticker in blacklist:
        ListOfTickers.remove(ticker)
    return ListOfTickers

In [19]:
# function to get the CIK of a company from the SEC database
def get_cik(ticker):
    ticker_symbol = ticker.upper()

    # getting all the companies ticker and CIK from the SEC database as a dictionary
    companyTickers = requests.get('https://www.sec.gov/files/company_tickers.json', headers={'User-Agent': "testing@gmail.com"}).json().values()

    # returns {'cik_str': 320193, 'ticker': 'AAPL', 'title': 'Apple Inc.'} for each company
    for company in companyTickers:
        if company['ticker'] == ticker_symbol:
            return(company)

    return('Ticker CIK not found in SEC database')

# function to get the SEC financial reports of a company
def sec_filings(ticker,metrics=['Assets']):

    ticker_symbol = ticker.upper()

    # grab CIK and fill leading zeroes
    cik = str(get_cik(ticker_symbol)['cik_str']).zfill(10)

    filingData = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers={'User-Agent': "testing@gmail.com"}).json()

    # used to check the key names for category of data
    # print(filingData['facts']['us-gaap'].keys())

    # this is how to grab the data from the json api
    metricList = []
    for metric in metrics:
        valList = []
        for i in filingData['facts']['us-gaap'][f'{metric}']['units']['USD']:
        # ['USD/shares']:
            val = i['val']
            valList.append(val)
        metricList.append(valList)

    return metricList
sec_filings('aapl')

[[39572000000,
  39572000000,
  36171000000,
  36171000000,
  48140000000,
  53851000000,
  47501000000,
  47501000000,
  47501000000,
  47501000000,
  47501000000,
  47501000000,
  53926000000,
  57057000000,
  64725000000,
  75183000000,
  75183000000,
  75183000000,
  75183000000,
  75183000000,
  86742000000,
  94904000000,
  106758000000,
  116371000000,
  116371000000,
  116371000000,
  116371000000,
  116371000000,
  116371000000,
  138681000000,
  150934000000,
  162896000000,
  176064000000,
  176064000000,
  176064000000,
  176064000000,
  176064000000,
  176064000000,
  196088000000,
  194743000000,
  199856000000,
  207000000000,
  207000000000,
  207000000000,
  207000000000,
  207000000000,
  207000000000,
  225184000000,
  205989000000,
  222520000000,
  231839000000,
  231839000000,
  231839000000,
  231839000000,
  231839000000,
  231839000000,
  261894000000,
  261194000000,
  273151000000,
  290479000000,
  290479000000,
  290479000000,
  290479000000,
  290345000000

In [25]:
def find_intersection(list1, list2):
    intersection = list(set(list1) & set(list2))
    return intersection

f = list(sec_filings('MMM'))
yuh =['NetCashProvidedByUsedInFinancingActivities', 'NetCashProvidedByUsedInInvestingActivities', 'Assets', 'LiabilitiesAndStockholdersEquity']

for ticker in ReadIndex():
    print(ticker)
    e = sec_filings(ticker,yuh)


   

MMM
AOS
ABT
ABBV
ACN
ADBE
AMD
AES
AFL
A
APD
ABNB
AKAM
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BK
BBWI
BAX
BDX
BBY
BIO
TECH
BIIB
BLK
BX
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BLDR
BG
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
COR
CNC
CNP
CF
CHRW
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CPAY
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHR
DRI
DVA
DAY
DECK
DE
DAL
DVN
DXCM
FANG
DLR
DFS
DG
DLTR
D
DPZ
DOV
DOW
DHI
DTE
DUK
DD
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FIS
FITB
FSLR
FE
FI
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GE
GEHC
GEV
GEN
GNRC
GD
GIS
GM
GPC
GILD
GPN
GL
GS
HAL
HIG
HAS
HCA
DOC
HSIC
HSY
HES
HPE
HLT
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUBB
HUM
HBA

In [16]:
cik = str(get_cik('WRK')['cik_str']).zfill(10)

filingData = requests.get(f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',headers={'User-Agent': "testing@gmail.com"}).json()

print(filingData['facts']['us-gaap'].keys())
# ['EarningsPerShareBasic']['units']['USD/shares']



TypeError: string indices must be integers, not 'str'

In [ ]:
val,filed = [],[]
val,filed = np.array(val), np.array(filed)
for i in filingData['facts']['us-gaap']['EarningsPerShareBasic']['units']['USD/shares']:
    val = np.append(i['val'],val)
    filed = np.append(i['filed'],filed)
    
val,filed = pd.DataFrame(val),pd.DataFrame(filed)

df = pd.concat([val,filed],axis=1)
df.columns = ['val','filed']
df = df.set_index(df['filed'])
df.drop(columns=['filed'],inplace=True)
df
